In [40]:
!pip install lda

import numpy as nt
import os, csv, nltk, lda
import pandas as pd
import numpy as np
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer



In [41]:
from nltk.tokenize import PunktSentenceTokenizer,RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

reviews_df=pd.read_excel("zara_results.xlsx")
reviews_list = reviews_df["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df['labels'] = reviews_list

(reviews_df)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Caption,Comments,Likes,URL,labels
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ..."
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'..."
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog..."
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'..."
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface..."
...,...,...,...,...,...
246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu..."
247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',..."
248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P..."
249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir..."


In [42]:

#checking for nulls if present any
print("Number of rows with any of the empty columns:")
print(reviews_df.isnull().sum().sum())
reviews_df=reviews_df.dropna()


restaurant_name = reviews_df.index

restaurant_name

Number of rows with any of the empty columns:
0


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            241, 242, 243, 244, 245, 246, 247, 248, 249, 250],
           dtype='int64', length=251)

In [43]:
# restaurant_name = input('provide the column name for id: ')
restaurant_review = input('provide the column name for text: ')
ntopics= input('Provide the number of latent topics: ');




provide the column name for text: labels
Provide the number of latent topics: 4


In [44]:

word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))


def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(reviews_df[restaurant_review])

print(total_features_words.shape)



(251, 409)


/Users/allymcnulty/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [45]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
reviews_df=reviews_df.join(doc_topic)
restaurant=pd.DataFrame()

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    restaurant[topic]=reviews_df.groupby([restaurant_name])[i].mean()

restaurant=restaurant.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
restaurant.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")





INFO:lda:n_documents: 251
INFO:lda:vocab_size: 409
INFO:lda:n_words: 3043
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -24867
INFO:lda:<10> log likelihood: -18046
INFO:lda:<20> log likelihood: -17377
INFO:lda:<30> log likelihood: -17072
INFO:lda:<40> log likelihood: -16885
INFO:lda:<50> log likelihood: -16841
INFO:lda:<60> log likelihood: -16740
INFO:lda:<70> log likelihood: -16709
INFO:lda:<80> log likelihood: -16680
INFO:lda:<90> log likelihood: -16669
INFO:lda:<100> log likelihood: -16581
INFO:lda:<110> log likelihood: -16558
INFO:lda:<120> log likelihood: -16598
INFO:lda:<130> log likelihood: -16667
INFO:lda:<140> log likelihood: -16566
INFO:lda:<150> log likelihood: -16605
INFO:lda:<160> log likelihood: -16568
INFO:lda:<170> log likelihood: -16596
INFO:lda:<180> log likelihood: -16643
INFO:lda:<190> log likelihood: -16601
INFO:lda:<200> log likelihood: -16569
INFO:lda:<210> log likelihood: -16602
INFO:lda:<220> log likelihood: -16618
INFO:lda:<230> log li

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


In [7]:
df_topic_zara = pd.read_excel('topic_word_dist.xlsx')
df_topic_dist_zara = pd.read_excel('document_topic_dist.xlsx')

df_topic_zara = df_topic_zara.rename(columns = {'Unnamed: 0': 'words', 0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

df_topic_zara_s = df_topic_zara.sort_values(by = 'topic_0', ascending = False)

df_topic_zara_s[:10]

## topic 1 is fashion-model


,words,topic_0,topic_1,topic_2,topic_3
131,fashion,0.067678,0.001653,0.000017,0.000017
324,sleeve,0.054940,0.000016,0.000017,0.000017
341,street,0.035037,0.000016,0.000017,0.000017
218,leg,0.035037,0.000016,0.000017,0.000017
107,dress,0.035037,0.000016,0.000017,0.000017
386,waist,0.035037,0.000016,0.000017,0.000017
96,design,0.026280,0.000016,0.000017,0.008419
361,thigh,0.025484,0.000016,0.000017,0.000017
396,white,0.024688,0.000016,0.013415,0.000017
193,human,0.024688,0.000016,0.011740,0.000017


In [8]:
df_topic_zara_s = df_topic_zara.sort_values(by = 'topic_1', ascending = False)

df_topic_zara_s[:10]

## topic 2 is nature



,words,topic_0,topic_1,topic_2,topic_3
177,happy,0.000008,0.060564,0.000017,0.000017
259,nature,0.000008,0.058927,0.000017,0.000017
281,people,0.000008,0.052382,0.000017,0.000017
288,photography,0.020707,0.052382,0.035187,0.001697
323,sky,0.005581,0.052382,0.000017,0.000017
140,flash,0.021503,0.040927,0.033513,0.000017
390,water,0.000008,0.024563,0.000017,0.008419
247,monochrome,0.000008,0.021290,0.000017,0.000017
118,entertainment,0.000008,0.021290,0.000017,0.000017
121,event,0.000008,0.019653,0.000017,0.000017


In [9]:
df_topic_zara_s = df_topic_zara.sort_values(by = 'topic_2', ascending = False)

df_topic_zara_s[:10]

## topic 3 is face

,words,topic_0,topic_1,topic_2,topic_3
227,lip,0.000008,0.000016,0.072033,0.000017
125,eyebrow,0.000008,0.000016,0.065334,0.000017
126,eyelash,0.000008,0.000016,0.061984,0.000017
172,hair,0.004785,0.000016,0.060309,0.000017
262,nose,0.000008,0.000016,0.051935,0.000017
150,forehead,0.000008,0.000016,0.045236,0.000017
322,skin,0.000008,0.000016,0.043561,0.000017
124,eye,0.000008,0.000016,0.040212,0.000017
288,photography,0.020707,0.052382,0.035187,0.001697
67,chin,0.000008,0.000016,0.035187,0.000017


In [10]:
df_topic_zara_s = df_topic_zara.sort_values(by = 'topic_3', ascending = False)

df_topic_zara_s[:10]

## topic 4 is colors

,words,topic_0,topic_1,topic_2,topic_3
148,font,0.000008,0.000016,0.000017,0.068914
38,blue,0.000008,0.000016,0.000017,0.050429
300,rectangle,0.000008,0.000016,0.000017,0.047069
10,art,0.000008,0.009835,0.000017,0.043708
115,electric,0.000008,0.000016,0.000017,0.040347
363,tints,0.000008,0.000016,0.000017,0.038666
311,shades,0.000008,0.000016,0.000017,0.038666
234,magenta,0.000008,0.000016,0.000017,0.021862
280,pattern,0.000008,0.000016,0.000017,0.021862
222,light,0.000008,0.000016,0.000017,0.020182


In [11]:
df_topic_dist_zara

,index,topic_0,topic_1,topic_2,topic_3
0,0,0.008772,0.096491,0.008772,0.885965
1,1,0.008065,0.169355,0.814516,0.008065
2,2,0.156716,0.380597,0.007463,0.455224
3,3,0.977612,0.007463,0.007463,0.007463
4,4,0.631944,0.076389,0.006944,0.284722
...,...,...,...,...,...
246,246,0.008772,0.973684,0.008772,0.008772
247,247,0.096491,0.359649,0.535088,0.008772
248,248,0.008772,0.973684,0.008772,0.008772
249,249,0.902985,0.082090,0.007463,0.007463


In [12]:

reviews_df

reviews_df = reviews_df.rename(columns = {0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",0.008772,0.096491,0.008772,0.885965
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",0.008065,0.169355,0.814516,0.008065
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",0.156716,0.380597,0.007463,0.455224
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",0.977612,0.007463,0.007463,0.007463
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",0.631944,0.076389,0.006944,0.284722
...,...,...,...,...,...,...,...,...,...
246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu...",0.008772,0.973684,0.008772,0.008772
247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',...",0.096491,0.359649,0.535088,0.008772
248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P...",0.008772,0.973684,0.008772,0.008772
249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir...",0.902985,0.082090,0.007463,0.007463


In [13]:
reviews_df.loc[:,('comment_norm')] = reviews_df['Comments']/max(reviews_df['Comments']) #normalizing comment count
reviews_df.loc[:,('likes_norm')] = reviews_df['Likes']/max(reviews_df['Likes']) #normalizing comment count


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",0.008772,0.096491,0.008772,0.885965,0.006154,0.093114
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",0.008065,0.169355,0.814516,0.008065,0.005137,0.192479
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",0.156716,0.380597,0.007463,0.455224,0.003318,0.067484
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",0.977612,0.007463,0.007463,0.007463,0.012201,0.153065
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",0.631944,0.076389,0.006944,0.284722,0.004415,0.137126
...,...,...,...,...,...,...,...,...,...,...,...
246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu...",0.008772,0.973684,0.008772,0.008772,0.006368,0.206394
247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',...",0.096491,0.359649,0.535088,0.008772,0.005030,0.182941
248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P...",0.008772,0.973684,0.008772,0.008772,0.004602,0.177369
249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir...",0.902985,0.082090,0.007463,0.007463,0.017097,0.399765


In [14]:
.4# likes (normalized) + .6# comments (normalized)

reviews_df.loc[:,('engage_scores')] = ((0.4* reviews_df.loc[:,('likes_norm')]) + 
                                 (0.6* reviews_df.loc[:,('comment_norm')]))


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",0.008772,0.096491,0.008772,0.885965,0.006154,0.093114,0.040938
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",0.008065,0.169355,0.814516,0.008065,0.005137,0.192479,0.080074
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",0.156716,0.380597,0.007463,0.455224,0.003318,0.067484,0.028984
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",0.977612,0.007463,0.007463,0.007463,0.012201,0.153065,0.068546
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",0.631944,0.076389,0.006944,0.284722,0.004415,0.137126,0.057499
...,...,...,...,...,...,...,...,...,...,...,...,...
246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu...",0.008772,0.973684,0.008772,0.008772,0.006368,0.206394,0.086378
247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',...",0.096491,0.359649,0.535088,0.008772,0.005030,0.182941,0.076195
248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P...",0.008772,0.973684,0.008772,0.008772,0.004602,0.177369,0.073709
249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir...",0.902985,0.082090,0.007463,0.007463,0.017097,0.399765,0.170164


In [15]:
q1=np.percentile(reviews_df.engage_scores, 25) 
q2=np.percentile(reviews_df.engage_scores, 50)  
q3=np.percentile(reviews_df.engage_scores, 75)

top_quartile = reviews_df[reviews_df['engage_scores']>q3]

top_quartile.sort_values(by ='engage_scores', ascending = False).head()

top_quartile_average_weights = top_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)

top_quartile_average_weights



topic_0    0.567872
topic_1    0.209291
topic_2    0.137931
topic_3    0.084905
dtype: float64

In [16]:
bottom_quartile = reviews_df[reviews_df['engage_scores']<q1]

bottom_quartile.sort_values(by ='engage_scores', ascending = True).head()

bottom_quartile_average_weights = bottom_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)


quartile_topics = pd.concat([top_quartile_average_weights,bottom_quartile_average_weights],axis=1)
quartile_topics.columns = ['Top Quartile','Bottom Quartile']

quartile_topics



,Top Quartile,Bottom Quartile
topic_0,0.567872,0.270195
topic_1,0.209291,0.174645
topic_2,0.137931,0.161800
topic_3,0.084905,0.393360


In [17]:
## Logistic regression on engagement and image features


# Assign values of 1 and 0 ...

center = reviews_df['engage_scores'].median()
#center
reviews_df['engagement_metr'] = np.where(reviews_df['engage_scores'] >= center, 1, 0)

# value counts of 1 and 0
reviews_df['engagement_metr'].value_counts()

# reviews_df.to_csv('metricscore.csv')

reviews_df


,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores,engagement_metr
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",0.008772,0.096491,0.008772,0.885965,0.006154,0.093114,0.040938,0
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",0.008065,0.169355,0.814516,0.008065,0.005137,0.192479,0.080074,1
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",0.156716,0.380597,0.007463,0.455224,0.003318,0.067484,0.028984,0
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",0.977612,0.007463,0.007463,0.007463,0.012201,0.153065,0.068546,0
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",0.631944,0.076389,0.006944,0.284722,0.004415,0.137126,0.057499,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu...",0.008772,0.973684,0.008772,0.008772,0.006368,0.206394,0.086378,1
247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',...",0.096491,0.359649,0.535088,0.008772,0.005030,0.182941,0.076195,0
248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P...",0.008772,0.973684,0.008772,0.008772,0.004602,0.177369,0.073709,0
249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir...",0.902985,0.082090,0.007463,0.007463,0.017097,0.399765,0.170164,1


In [18]:
import pandas as pd
from pandas import DataFrame, Series
import urllib.request 
import statsmodels.api as sm
import math
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

In [19]:
reviews_df_copy = reviews_df.copy()

reviews_df_copy['labels_str'] = reviews_df_copy['labels'].apply(tokenize_text)

reviews_df_copy['labels_str'] = reviews_df_copy['labels_str'].replace("'","")

reviews_df_copy['labels_str']

def list_to_string(s):
    str1 = " "
    return (str1.join(s))

reviews_df_copy['labels_str'] = reviews_df_copy['labels_str'].apply(list_to_string)

reviews_df_copy['labels_str']

reviews_df_copy

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores,engagement_metr,labels_str
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",0.008772,0.096491,0.008772,0.885965,0.006154,0.093114,0.040938,0,rectangle font technology electric blue circle...
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",0.008065,0.169355,0.814516,0.008065,0.005137,0.192479,0.080074,1,nose hair lip outerwear eyebrow eyelash flash ...
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",0.156716,0.380597,0.007463,0.455224,0.003318,0.067484,0.028984,0,black and white font monochrome photography mo...
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",0.977612,0.007463,0.007463,0.007463,0.012201,0.153065,0.068546,0,arm window photograph leg white black human bo...
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",0.631944,0.076389,0.006944,0.284722,0.004415,0.137126,0.057499,0,light automotive lighting road surface black h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu...",0.008772,0.973684,0.008772,0.008772,0.006368,0.206394,0.086378,1,sky dinosaur people in nature gesture happy gr...
247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',...",0.096491,0.359649,0.535088,0.008772,0.005030,0.182941,0.076195,0,hair head sky arm eye mouth nature natural env...
248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P...",0.008772,0.973684,0.008772,0.008772,0.004602,0.177369,0.073709,0,sky smile vertebrate ecoregion people in natur...
249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir...",0.902985,0.082090,0.007463,0.007463,0.017097,0.399765,0.170164,1,clothing outerwear water dress shirt sky stree...


In [20]:
reviews_df_copy = reviews_df_copy.dropna(subset = ['labels_str']).copy()


reviews_df_copy['labels_str'] = reviews_df_copy['labels_str'].apply(lambda x: x.lower())


from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
reviews_df_copy['labels_str'].isnull().values.any()

reviews_df_copy = reviews_df_copy.reset_index()


vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_copy['labels_str'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagement_metr'] = reviews_df_copy['engagement_metr']


X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagement_metr'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train,y_train )
print("Accuracy with Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))

l

Accuracy with Image Labels: 0.631578947368421
Confusion Matrix:
[[11  9]
 [ 5 13]]


,abdomen,accessory,active,adaptation,aeolian,amber,ancient,and,aqua,arm,...,wood,worker,workwear,world,wrap,wrist,writing,yellow,zebra,engagement_metr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.172727,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.207614,0.0,0.251349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.155536,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327919,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1
247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.291818,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1


In [22]:
# Keeping all captions intact and using tf-idf score
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_copy['Caption'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = reviews_df_copy['engagement_metr']

X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with Caption Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))



Accuracy with Caption Labels: 0.6578947368421053
Confusion Matrix:
[[11  9]
 [ 4 14]]


In [23]:
reviews_df_copy['Caption+labels'] = reviews_df_copy['Caption'] + " " + reviews_df_copy['labels_str']

reviews_df_copy

,index,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores,engagement_metr,labels_str,Caption+labels
0,0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",0.008772,0.096491,0.008772,0.885965,0.006154,0.093114,0.040938,0,rectangle font technology electric blue circle...,The Charlotte Gainsbourg Collection for Zara. ...
1,1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",0.008065,0.169355,0.814516,0.008065,0.005137,0.192479,0.080074,1,nose hair lip outerwear eyebrow eyelash flash ...,The Charlotte Gainsbourg Collection for Zara....
2,2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",0.156716,0.380597,0.007463,0.455224,0.003318,0.067484,0.028984,0,black and white font monochrome photography mo...,The Charlotte Gainsbourg Collection for Zara....
3,3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",0.977612,0.007463,0.007463,0.007463,0.012201,0.153065,0.068546,0,arm window photograph leg white black human bo...,ZARA ATHLETICZ. Active wear that goes faster a...
4,4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",0.631944,0.076389,0.006944,0.284722,0.004415,0.137126,0.057499,0,light automotive lighting road surface black h...,"ZARA ATHLETICZ. Adaptable, multifunctional des..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,246,Orange vibes @zarakids,238,52302,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Dinosaur', 'People in nature', 'Gestu...",0.008772,0.973684,0.008772,0.008772,0.006368,0.206394,0.086378,1,sky dinosaur people in nature gesture happy gr...,Orange vibes @zarakids sky dinosaur people in ...
247,247,Orange vibes @zarakids,188,46359,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Head', 'Sky', 'Arm', 'Eye', 'Mouth',...",0.096491,0.359649,0.535088,0.008772,0.005030,0.182941,0.076195,0,hair head sky arm eye mouth nature natural env...,Orange vibes @zarakids hair head sky arm eye m...
248,248,Orange vibes @zarakids,172,44947,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Smile', 'Vertebrate', 'Ecoregion', 'P...",0.008772,0.973684,0.008772,0.008772,0.004602,0.177369,0.073709,0,sky smile vertebrate ecoregion people in natur...,Orange vibes @zarakids sky smile vertebrate ec...
249,249,Print shirt #zarawoman,639,101304,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Outerwear', 'Water', 'Dress shir...",0.902985,0.082090,0.007463,0.007463,0.017097,0.399765,0.170164,1,clothing outerwear water dress shirt sky stree...,Print shirt #zarawoman clothing outerwear wate...


In [24]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_copy['Caption+labels'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = reviews_df_copy['engagement_metr']

X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with both Captions and Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))





Accuracy with both Captions and Image Labels: 0.6578947368421053
Confusion Matrix:
[[10 10]
 [ 3 15]]


In [25]:
## HM

reviews_df_hm=pd.read_excel("hm_results.xlsx")
reviews_list = reviews_df_hm["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df_hm['labels'] = reviews_list

reviews_df_hm.loc[:,('comment_norm')] = reviews_df_hm['Comments']/max(reviews_df_hm['Comments']) #normalizing comment count
reviews_df_hm.loc[:,('likes_norm')] = reviews_df_hm['Likes']/max(reviews_df_hm['Likes']) #normalizing comment count


reviews_df_hm.loc[:,('engage_scores')] = ((0.4* reviews_df_hm.loc[:,('likes_norm')]) + 
                                 (0.6* reviews_df_hm.loc[:,('comment_norm')]))


center = reviews_df_hm['engage_scores'].median()
#center
reviews_df_hm['engagement_metr'] = np.where(reviews_df_hm['engage_scores'] >= center, 1, 0)

# value counts of 1 and 0
reviews_df_hm['engagement_metr'].value_counts()

reviews_df_hm

,Caption,Comments,Likes,URL,labels,comment_norm,likes_norm,engage_scores,engagement_metr
0,Lean on @valerialipovetsky for some comfy/casu...,115,48549,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Joint', 'Shoe', 'Plant', 'Should...",0.071740,0.249192,0.142721,1
1,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...,279,51340,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Shoe', 'Durango boot', 'Beige', ...",0.174049,0.263517,0.209836,1
2,BIG NEWS! Award-winning actor @johnboyega has ...,37,13909,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Forehead', 'Chin', 'Hairstyle', 'Jaw', 'Neck...",0.023082,0.071392,0.042406,0
3,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...,109,36864,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Face', 'Head', 'Smile', 'Shoe', ...",0.067998,0.189215,0.116484,0
4,All you need for fall.🍂 #HM\n\nRibbed polo-nec...,224,71673,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Comfort', 'Wood', 'Tints and shades', 'Work ...",0.139738,0.367882,0.230996,1
...,...,...,...,...,...,...,...,...,...
246,Stay fresh in mesh like @sintijaselicka #HM #H...,154,47547,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Plant', 'Azure', 'Window', 'People in...",0.096070,0.244049,0.155261,1
247,A swimsuit that doubles as a statement look? S...,84,42667,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Leg', 'Waist', 'Thigh', 'Flash photography',...",0.052402,0.219001,0.119041,0
248,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...,106,38529,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Shorts', 'Window', 'Stomach', 'Flas...",0.066126,0.197761,0.118780,0
249,Orange you glad this playsuit exists? 🧡 @ssolo...,119,43813,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Smile', 'Azure', 'Human', 'Street fashion', ...",0.074236,0.224883,0.134495,0


In [26]:

reviews_df_hm_copy = reviews_df_hm.copy()

reviews_df_hm_copy['labels_str'] = reviews_df_hm_copy['labels'].apply(tokenize_text)

reviews_df_hm_copy['labels_str'] = reviews_df_hm_copy['labels_str'].replace("'","")

reviews_df_hm_copy['labels_str']

def list_to_string(s):
    str1 = " "
    return (str1.join(s))

reviews_df_hm_copy['labels_str'] = reviews_df_hm_copy['labels_str'].apply(list_to_string)

reviews_df_hm_copy['labels_str']

reviews_df_hm_copy['Caption+labels'] = reviews_df_hm_copy['Caption'] + " " + reviews_df_hm_copy['labels_str']


reviews_df_hm_copy



,Caption,Comments,Likes,URL,labels,comment_norm,likes_norm,engage_scores,engagement_metr,labels_str,Caption+labels
0,Lean on @valerialipovetsky for some comfy/casu...,115,48549,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Joint', 'Shoe', 'Plant', 'Should...",0.071740,0.249192,0.142721,1,footwear joint shoe plant shoulder one piece g...,Lean on @valerialipovetsky for some comfy/casu...
1,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...,279,51340,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Shoe', 'Durango boot', 'Beige', ...",0.174049,0.263517,0.209836,1,footwear shoe durango boot beige boot font wal...,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...
2,BIG NEWS! Award-winning actor @johnboyega has ...,37,13909,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Forehead', 'Chin', 'Hairstyle', 'Jaw', 'Neck...",0.023082,0.071392,0.042406,0,forehead chin hairstyle jaw neck gesture dress...,BIG NEWS! Award-winning actor @johnboyega has ...
3,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...,109,36864,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Face', 'Head', 'Smile', 'Shoe', ...",0.067998,0.189215,0.116484,0,clothing face head smile shoe outerwear photog...,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...
4,All you need for fall.🍂 #HM\n\nRibbed polo-nec...,224,71673,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Comfort', 'Wood', 'Tints and shades', 'Work ...",0.139738,0.367882,0.230996,1,comfort wood tints and shades work boots autom...,All you need for fall.🍂 #HM\n\nRibbed polo-nec...
...,...,...,...,...,...,...,...,...,...,...,...
246,Stay fresh in mesh like @sintijaselicka #HM #H...,154,47547,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Plant', 'Azure', 'Window', 'People in...",0.096070,0.244049,0.155261,1,sky plant azure window people in nature eyewea...,Stay fresh in mesh like @sintijaselicka #HM #H...
247,A swimsuit that doubles as a statement look? S...,84,42667,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Leg', 'Waist', 'Thigh', 'Flash photography',...",0.052402,0.219001,0.119041,0,leg waist thigh flash photography comfort floo...,A swimsuit that doubles as a statement look? S...
248,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...,106,38529,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Shorts', 'Window', 'Stomach', 'Flas...",0.066126,0.197761,0.118780,0,joint shorts window stomach flash photography ...,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...
249,Orange you glad this playsuit exists? 🧡 @ssolo...,119,43813,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Smile', 'Azure', 'Human', 'Street fashion', ...",0.074236,0.224883,0.134495,0,smile azure human street fashion fashion templ...,Orange you glad this playsuit exists? 🧡 @ssolo...


In [27]:
reviews_df_hm_copy = reviews_df_hm_copy.dropna(subset = ['labels_str']).copy()


reviews_df_hm_copy['labels_str'] = reviews_df_hm_copy['labels_str'].apply(lambda x: x.lower())


from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

reviews_df_hm_copy['labels_str'].isnull().values.any()

reviews_df_hm_copy = reviews_df_hm_copy.reset_index()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_hm_copy['labels_str'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = reviews_df_hm_copy['engagement_metr']


X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy with Image Labels: 0.5789473684210527
Confusion Matrix:
[[ 9  7]
 [ 9 13]]


In [28]:
## GUCCI

reviews_df_gucci=pd.read_excel("gucci_results.xlsx")
reviews_list = reviews_df_gucci["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df_gucci['labels'] = reviews_list

reviews_df_gucci.loc[:,('comment_norm')] = reviews_df_gucci['Comments']/max(reviews_df_gucci['Comments']) #normalizing comment count
reviews_df_gucci.loc[:,('likes_norm')] = reviews_df_gucci['Likes']/max(reviews_df_gucci['Likes']) #normalizing comment count


reviews_df_gucci.loc[:,('engage_scores')] = ((0.4* reviews_df_gucci.loc[:,('likes_norm')]) + 
                                 (0.6* reviews_df_gucci.loc[:,('comment_norm')]))


center = reviews_df_gucci['engage_scores'].median()
#center
reviews_df_gucci['engagement_metr'] = np.where(reviews_df_gucci['engage_scores'] >= center, 1, 0)

# value counts of 1 and 0
reviews_df_gucci['engagement_metr'].value_counts()

reviews_df_gucci



,Caption,Comments,Likes,URL,labels,comment_norm,likes_norm,engage_scores,engagement_metr
0,"Part of its global tour, the Gucci Garden Arch...",86,16095,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Building', 'Font', 'Cloud', 'Gas', 'S...",0.018151,0.023688,0.020366,0
1,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats...",100,35382,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Magenta', 'Fashion design', 'Poster', 'Toy',...",0.021106,0.052074,0.033493,0
2,"@elleusa’s October cover star, @billieeilish, ...",159,100026,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hairstyle', 'Black', 'Vision care', ...",0.033558,0.147216,0.079022,0
3,The October cover of L'Officiel Hommes China f...,133,80664,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Jeans', 'Head', 'Lip', 'Sleeve', 'Wa...",0.028071,0.118720,0.064330,0
4,@chiaraferragni appears on the cover of @vogue...,214,89919,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Outerwear', 'Arm', 'One-piece garment', 'Smi...",0.045167,0.132341,0.080036,0
...,...,...,...,...,...,...,...,...,...
246,"Evoking Hollywood glamour, @jodiesmith stars i...",280,48181,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Body jewelry', 'Neck', 'Eyelash', 'Headgear'...",0.059097,0.070912,0.063823,0
247,"Launching soon on @guccibeauty, a joy-fueled f...",446,120213,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Cheek', 'Lip', 'Sky', 'Eyelash', 'Hu...",0.094133,0.176927,0.127250,1
248,"First unveiled on @guccibeauty, multi-platinum...",736,89705,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Skin', 'Lip', 'Chin', 'Hand', 'Flowe...",0.155340,0.132026,0.146014,1
249,Have you heard? @guccibeauty has a new face. S...,431,125333,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Skin', 'Lip', 'Neck', 'Carnivore', 'Gesture'...",0.090967,0.184462,0.128365,1


In [29]:

reviews_df_gucci_copy = reviews_df_gucci.copy()

reviews_df_gucci_copy['labels_str'] = reviews_df_gucci_copy['labels'].apply(tokenize_text)

reviews_df_gucci_copy['labels_str'] = reviews_df_gucci_copy['labels_str'].replace("'","")

reviews_df_gucci_copy['labels_str']

def list_to_string(s):
    str1 = " "
    return (str1.join(s))

reviews_df_gucci_copy['labels_str'] = reviews_df_gucci_copy['labels_str'].apply(list_to_string)

reviews_df_gucci_copy['labels_str']

reviews_df_gucci_copy['Caption+labels'] = reviews_df_gucci_copy['Caption'] + " " + reviews_df_gucci_copy['labels_str']


reviews_df_gucci_copy




,Caption,Comments,Likes,URL,labels,comment_norm,likes_norm,engage_scores,engagement_metr,labels_str,Caption+labels
0,"Part of its global tour, the Gucci Garden Arch...",86,16095,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Building', 'Font', 'Cloud', 'Gas', 'S...",0.018151,0.023688,0.020366,0,sky building font cloud gas signage street lig...,"Part of its global tour, the Gucci Garden Arch..."
1,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats...",100,35382,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Magenta', 'Fashion design', 'Poster', 'Toy',...",0.021106,0.052074,0.033493,0,magenta fashion design poster toy event art ad...,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats..."
2,"@elleusa’s October cover star, @billieeilish, ...",159,100026,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hairstyle', 'Black', 'Vision care', ...",0.033558,0.147216,0.079022,0,face hairstyle black vision care eyelash black...,"@elleusa’s October cover star, @billieeilish, ..."
3,The October cover of L'Officiel Hommes China f...,133,80664,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Jeans', 'Head', 'Lip', 'Sleeve', 'Wa...",0.028071,0.118720,0.064330,0,hair jeans head lip sleeve waist thigh street ...,The October cover of L'Officiel Hommes China f...
4,@chiaraferragni appears on the cover of @vogue...,214,89919,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Outerwear', 'Arm', 'One-piece garment', 'Smi...",0.045167,0.132341,0.080036,0,outerwear arm one piece garment smile dress ne...,@chiaraferragni appears on the cover of @vogue...
...,...,...,...,...,...,...,...,...,...,...,...
246,"Evoking Hollywood glamour, @jodiesmith stars i...",280,48181,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Body jewelry', 'Neck', 'Eyelash', 'Headgear'...",0.059097,0.070912,0.063823,0,body jewelry neck eyelash headgear necklace fa...,"Evoking Hollywood glamour, @jodiesmith stars i..."
247,"Launching soon on @guccibeauty, a joy-fueled f...",446,120213,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Cheek', 'Lip', 'Sky', 'Eyelash', 'Hu...",0.094133,0.176927,0.127250,1,nose cheek lip sky eyelash human body flash ph...,"Launching soon on @guccibeauty, a joy-fueled f..."
248,"First unveiled on @guccibeauty, multi-platinum...",736,89705,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Skin', 'Lip', 'Chin', 'Hand', 'Flowe...",0.155340,0.132026,0.146014,1,nose skin lip chin hand flower eyelash mouth h...,"First unveiled on @guccibeauty, multi-platinum..."
249,Have you heard? @guccibeauty has a new face. S...,431,125333,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Skin', 'Lip', 'Neck', 'Carnivore', 'Gesture'...",0.090967,0.184462,0.128365,1,skin lip neck carnivore gesture finger cat nai...,Have you heard? @guccibeauty has a new face. S...


In [30]:
reviews_df_gucci_copy = reviews_df_gucci_copy.dropna(subset = ['labels_str']).copy()


reviews_df_gucci_copy['labels_str'] = reviews_df_gucci_copy['labels_str'].apply(lambda x: x.lower())


from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

reviews_df_gucci_copy['labels_str'].isnull().values.any()

reviews_df_gucci_copy = reviews_df_gucci_copy.reset_index()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_gucci_copy['labels_str'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = reviews_df_gucci_copy['engagement_metr']


X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))





[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy with Image Labels: 0.5
Confusion Matrix:
[[ 9 10]
 [ 9 10]]


In [31]:
reviews_df_gucci_copy = reviews_df_gucci_copy.dropna(subset = ['Caption']).copy()


reviews_df_gucci_copy['Caption'] = reviews_df_gucci_copy['Caption'].apply(lambda x: x.lower())


from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

reviews_df_gucci_copy['Caption'].isnull().values.any()

reviews_df_gucci_copy = reviews_df_gucci_copy.reset_index()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_gucci_copy['Caption'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = reviews_df_gucci_copy['engagement_metr']


X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with Caption Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))






[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Accuracy with Caption Labels: 0.7368421052631579
Confusion Matrix:
[[19  0]
 [10  9]]


In [32]:
reviews_df_gucci_copy = reviews_df_gucci_copy.dropna(subset = ['Caption+labels']).copy()


reviews_df_gucci_copy['Caption+labels'] = reviews_df_gucci_copy['Caption+labels'].apply(lambda x: x.lower())


from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')

reviews_df_gucci_copy['Caption+labels'].isnull().values.any()

reviews_df_gucci_copy = reviews_df_gucci_copy.reset_index()

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(reviews_df_gucci_copy['Caption+labels'].tolist())
q = vectorizer.get_feature_names()
l = pd.DataFrame(X.toarray())
for i in range(len(q)):
    l = l.rename(columns={i: q[i]}) 

l['engagementVal'] = reviews_df_gucci_copy['engagement_metr']


X_train, X_test, y_train, y_test = train_test_split(l.iloc[:,:-1], l['engagementVal'], test_size=0.15, random_state=42)

clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X_train,y_train )
print("Accuracy with Caption and Image Labels: " + str(clf.score(X_test, y_test)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, clf.predict(X_test)))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/allymcnulty/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ValueError: cannot insert level_0, already exists

## HM Topic Modeling

In [46]:
reviews_df=pd.read_excel("hm_results.xlsx")
reviews_list = reviews_df["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df['labels'] = reviews_list

(reviews_df)

,Caption,Comments,Likes,URL,labels
0,Lean on @valerialipovetsky for some comfy/casu...,115,48549,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Joint', 'Shoe', 'Plant', 'Should..."
1,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...,279,51340,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Shoe', 'Durango boot', 'Beige', ..."
2,BIG NEWS! Award-winning actor @johnboyega has ...,37,13909,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Forehead', 'Chin', 'Hairstyle', 'Jaw', 'Neck..."
3,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...,109,36864,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Face', 'Head', 'Smile', 'Shoe', ..."
4,All you need for fall.🍂 #HM\n\nRibbed polo-nec...,224,71673,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Comfort', 'Wood', 'Tints and shades', 'Work ..."
...,...,...,...,...,...
246,Stay fresh in mesh like @sintijaselicka #HM #H...,154,47547,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Plant', 'Azure', 'Window', 'People in..."
247,A swimsuit that doubles as a statement look? S...,84,42667,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Leg', 'Waist', 'Thigh', 'Flash photography',..."
248,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...,106,38529,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Shorts', 'Window', 'Stomach', 'Flas..."
249,Orange you glad this playsuit exists? 🧡 @ssolo...,119,43813,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Smile', 'Azure', 'Human', 'Street fashion', ..."


In [47]:
#checking for nulls if present any
print("Number of rows with any of the empty columns:")
print(reviews_df.isnull().sum().sum())
reviews_df=reviews_df.dropna()


restaurant_name = reviews_df.index

restaurant_name

Number of rows with any of the empty columns:
0


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            241, 242, 243, 244, 245, 246, 247, 248, 249, 250],
           dtype='int64', length=251)

In [48]:
# restaurant_name = input('provide the column name for id: ')
restaurant_review = input('provide the column name for text: ')
ntopics= input('Provide the number of latent topics: ');

provide the column name for text: labels
Provide the number of latent topics: 4


In [49]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))


def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(reviews_df[restaurant_review])

print(total_features_words.shape)


(251, 319)


/Users/allymcnulty/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [50]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
reviews_df=reviews_df.join(doc_topic)
restaurant=pd.DataFrame()

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    restaurant[topic]=reviews_df.groupby([restaurant_name])[i].mean()

restaurant=restaurant.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
restaurant.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")

INFO:lda:n_documents: 251
INFO:lda:vocab_size: 319
INFO:lda:n_words: 3071
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -23228
INFO:lda:<10> log likelihood: -16950
INFO:lda:<20> log likelihood: -16474
INFO:lda:<30> log likelihood: -16303
INFO:lda:<40> log likelihood: -16082
INFO:lda:<50> log likelihood: -15948
INFO:lda:<60> log likelihood: -15893
INFO:lda:<70> log likelihood: -15945
INFO:lda:<80> log likelihood: -15931
INFO:lda:<90> log likelihood: -15838
INFO:lda:<100> log likelihood: -15861
INFO:lda:<110> log likelihood: -15799
INFO:lda:<120> log likelihood: -15818
INFO:lda:<130> log likelihood: -15821
INFO:lda:<140> log likelihood: -15744
INFO:lda:<150> log likelihood: -15693
INFO:lda:<160> log likelihood: -15772
INFO:lda:<170> log likelihood: -15711
INFO:lda:<180> log likelihood: -15747
INFO:lda:<190> log likelihood: -15779
INFO:lda:<200> log likelihood: -15742
INFO:lda:<210> log likelihood: -15673
INFO:lda:<220> log likelihood: -15725
INFO:lda:<230> log li

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


In [51]:
## hm topic modeling
df_topic_hm = pd.read_excel('topic_word_dist.xlsx')
df_topic_dist_hm = pd.read_excel('document_topic_dist.xlsx')

df_topic_hm = df_topic_hm.rename(columns = {'Unnamed: 0': 'words', 0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

df_topic_hm_s = df_topic_hm.sort_values(by = 'topic_0', ascending = False)

df_topic_hm_s[:10]

## topic 1 materials


,words,topic_0,topic_1,topic_2,topic_3
21,bag,0.054919,0.000011,0.000014,0.000010
65,comfort,0.045770,0.007779,0.000014,0.000010
251,sleeve,0.041195,0.017765,0.069010,0.065698
312,wood,0.041195,0.004450,0.000014,0.000010
77,design,0.038908,0.000011,0.052113,0.000010
243,shoe,0.034333,0.000011,0.000014,0.025126
282,textile,0.029758,0.000011,0.029584,0.007738
110,flooring,0.027471,0.000011,0.000014,0.000010
113,font,0.027471,0.000011,0.008463,0.000010
28,beige,0.025184,0.000011,0.000014,0.000010


In [53]:
df_topic_hm_s = df_topic_hm.sort_values(by = 'topic_1', ascending = False)

df_topic_hm_s[:10]

## topic 2 is photography

,words,topic_0,topic_1,topic_2,topic_3
108,flash,0.000023,0.046616,0.000014,0.000010
215,photography,0.000023,0.046616,0.000014,0.000010
209,people,0.000023,0.043287,0.000014,0.000010
190,nature,0.000023,0.039958,0.000014,0.000010
245,shoulder,0.000023,0.036629,0.000014,0.046378
130,hair,0.000023,0.034410,0.000014,0.000010
135,happy,0.000023,0.029971,0.000014,0.000010
250,sky,0.000023,0.029971,0.000014,0.007738
301,waist,0.000023,0.028862,0.018319,0.066664
216,piece,0.000023,0.025533,0.000014,0.000010


In [54]:
df_topic_hm_s = df_topic_hm.sort_values(by = 'topic_2', ascending = False)

df_topic_hm_s[:10]

## topic 3 is fashion model

,words,topic_0,topic_1,topic_2,topic_3
102,fashion,0.009172,0.000011,0.128149,0.079222
251,sleeve,0.041195,0.017765,0.069010,0.065698
83,dress,0.000023,0.024423,0.054929,0.016432
264,street,0.000023,0.000011,0.053521,0.062800
77,design,0.038908,0.000011,0.052113,0.000010
242,shirt,0.000023,0.000011,0.052113,0.013534
198,outerwear,0.000023,0.000011,0.043664,0.023194
64,collar,0.000023,0.000011,0.040848,0.000010
282,textile,0.029758,0.000011,0.029584,0.007738
191,neck,0.000023,0.014436,0.023951,0.042514


In [55]:
df_topic_hm_s = df_topic_hm.sort_values(by = 'topic_3', ascending = False)

df_topic_hm_s[:10]

## topic 4 is body

,words,topic_0,topic_1,topic_2,topic_3
102,fashion,0.009172,0.000011,0.128149,0.079222
301,waist,0.000023,0.028862,0.018319,0.066664
251,sleeve,0.041195,0.017765,0.069010,0.065698
264,street,0.000023,0.000011,0.053521,0.062800
245,shoulder,0.000023,0.036629,0.000014,0.046378
191,neck,0.000023,0.014436,0.023951,0.042514
99,eyewear,0.000023,0.000011,0.009871,0.037684
164,leg,0.018322,0.022204,0.000014,0.037684
267,sunglasses,0.000023,0.000011,0.000014,0.034786
50,care,0.000023,0.000011,0.000014,0.028024


In [57]:
df_topic_dist_hm

,index,topic_0,topic_1,topic_2,topic_3
0,0,0.008065,0.411290,0.008065,0.572581
1,1,0.915584,0.006494,0.006494,0.071429
2,2,0.007463,0.529851,0.231343,0.231343
3,3,0.009615,0.586538,0.009615,0.394231
4,4,0.981707,0.006098,0.006098,0.006098
...,...,...,...,...,...
246,246,0.008772,0.798246,0.008772,0.184211
247,247,0.169355,0.411290,0.008065,0.411290
248,248,0.008772,0.271930,0.008772,0.710526
249,249,0.008772,0.008772,0.008772,0.973684


In [58]:
reviews_df

reviews_df = reviews_df.rename(columns = {0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3
0,Lean on @valerialipovetsky for some comfy/casu...,115,48549,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Joint', 'Shoe', 'Plant', 'Should...",0.008065,0.411290,0.008065,0.572581
1,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...,279,51340,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Shoe', 'Durango boot', 'Beige', ...",0.915584,0.006494,0.006494,0.071429
2,BIG NEWS! Award-winning actor @johnboyega has ...,37,13909,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Forehead', 'Chin', 'Hairstyle', 'Jaw', 'Neck...",0.007463,0.529851,0.231343,0.231343
3,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...,109,36864,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Face', 'Head', 'Smile', 'Shoe', ...",0.009615,0.586538,0.009615,0.394231
4,All you need for fall.🍂 #HM\n\nRibbed polo-nec...,224,71673,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Comfort', 'Wood', 'Tints and shades', 'Work ...",0.981707,0.006098,0.006098,0.006098
...,...,...,...,...,...,...,...,...,...
246,Stay fresh in mesh like @sintijaselicka #HM #H...,154,47547,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Plant', 'Azure', 'Window', 'People in...",0.008772,0.798246,0.008772,0.184211
247,A swimsuit that doubles as a statement look? S...,84,42667,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Leg', 'Waist', 'Thigh', 'Flash photography',...",0.169355,0.411290,0.008065,0.411290
248,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...,106,38529,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Shorts', 'Window', 'Stomach', 'Flas...",0.008772,0.271930,0.008772,0.710526
249,Orange you glad this playsuit exists? 🧡 @ssolo...,119,43813,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Smile', 'Azure', 'Human', 'Street fashion', ...",0.008772,0.008772,0.008772,0.973684


In [59]:
reviews_df.loc[:,('comment_norm')] = reviews_df['Comments']/max(reviews_df['Comments']) #normalizing comment count
reviews_df.loc[:,('likes_norm')] = reviews_df['Likes']/max(reviews_df['Likes']) #normalizing comment count


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm
0,Lean on @valerialipovetsky for some comfy/casu...,115,48549,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Joint', 'Shoe', 'Plant', 'Should...",0.008065,0.411290,0.008065,0.572581,0.071740,0.249192
1,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...,279,51340,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Shoe', 'Durango boot', 'Beige', ...",0.915584,0.006494,0.006494,0.071429,0.174049,0.263517
2,BIG NEWS! Award-winning actor @johnboyega has ...,37,13909,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Forehead', 'Chin', 'Hairstyle', 'Jaw', 'Neck...",0.007463,0.529851,0.231343,0.231343,0.023082,0.071392
3,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...,109,36864,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Face', 'Head', 'Smile', 'Shoe', ...",0.009615,0.586538,0.009615,0.394231,0.067998,0.189215
4,All you need for fall.🍂 #HM\n\nRibbed polo-nec...,224,71673,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Comfort', 'Wood', 'Tints and shades', 'Work ...",0.981707,0.006098,0.006098,0.006098,0.139738,0.367882
...,...,...,...,...,...,...,...,...,...,...,...
246,Stay fresh in mesh like @sintijaselicka #HM #H...,154,47547,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Plant', 'Azure', 'Window', 'People in...",0.008772,0.798246,0.008772,0.184211,0.096070,0.244049
247,A swimsuit that doubles as a statement look? S...,84,42667,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Leg', 'Waist', 'Thigh', 'Flash photography',...",0.169355,0.411290,0.008065,0.411290,0.052402,0.219001
248,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...,106,38529,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Shorts', 'Window', 'Stomach', 'Flas...",0.008772,0.271930,0.008772,0.710526,0.066126,0.197761
249,Orange you glad this playsuit exists? 🧡 @ssolo...,119,43813,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Smile', 'Azure', 'Human', 'Street fashion', ...",0.008772,0.008772,0.008772,0.973684,0.074236,0.224883


In [60]:

reviews_df.loc[:,('engage_scores')] = ((0.4* reviews_df.loc[:,('likes_norm')]) + 
                                 (0.6* reviews_df.loc[:,('comment_norm')]))


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores
0,Lean on @valerialipovetsky for some comfy/casu...,115,48549,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Joint', 'Shoe', 'Plant', 'Should...",0.008065,0.411290,0.008065,0.572581,0.071740,0.249192,0.142721
1,So fresh and so clean 👢✨ #HM \n\nCorduroy trou...,279,51340,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Footwear', 'Shoe', 'Durango boot', 'Beige', ...",0.915584,0.006494,0.006494,0.071429,0.174049,0.263517,0.209836
2,BIG NEWS! Award-winning actor @johnboyega has ...,37,13909,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Forehead', 'Chin', 'Hairstyle', 'Jaw', 'Neck...",0.007463,0.529851,0.231343,0.231343,0.023082,0.071392,0.042406
3,TFW elevated gorpcore meets 90s athleisure❣️🍁🥾...,109,36864,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Face', 'Head', 'Smile', 'Shoe', ...",0.009615,0.586538,0.009615,0.394231,0.067998,0.189215,0.116484
4,All you need for fall.🍂 #HM\n\nRibbed polo-nec...,224,71673,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Comfort', 'Wood', 'Tints and shades', 'Work ...",0.981707,0.006098,0.006098,0.006098,0.139738,0.367882,0.230996
...,...,...,...,...,...,...,...,...,...,...,...,...
246,Stay fresh in mesh like @sintijaselicka #HM #H...,154,47547,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Plant', 'Azure', 'Window', 'People in...",0.008772,0.798246,0.008772,0.184211,0.096070,0.244049,0.155261
247,A swimsuit that doubles as a statement look? S...,84,42667,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Leg', 'Waist', 'Thigh', 'Flash photography',...",0.169355,0.411290,0.008065,0.411290,0.052402,0.219001,0.119041
248,Chill Sunday vibes: ACTIVATED. ☑️\n@chankimber...,106,38529,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Shorts', 'Window', 'Stomach', 'Flas...",0.008772,0.271930,0.008772,0.710526,0.066126,0.197761,0.118780
249,Orange you glad this playsuit exists? 🧡 @ssolo...,119,43813,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Smile', 'Azure', 'Human', 'Street fashion', ...",0.008772,0.008772,0.008772,0.973684,0.074236,0.224883,0.134495


In [61]:
q1=np.percentile(reviews_df.engage_scores, 25) 
q2=np.percentile(reviews_df.engage_scores, 50)  
q3=np.percentile(reviews_df.engage_scores, 75)

top_quartile = reviews_df[reviews_df['engage_scores']>q3]

top_quartile.sort_values(by ='engage_scores', ascending = False).head()

top_quartile_average_weights = top_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)

top_quartile_average_weights

topic_0    0.202399
topic_1    0.182921
topic_2    0.260521
topic_3    0.354159
dtype: float64

In [62]:
bottom_quartile = reviews_df[reviews_df['engage_scores']<q1]

bottom_quartile.sort_values(by ='engage_scores', ascending = True).head()

bottom_quartile_average_weights = bottom_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)


quartile_topics = pd.concat([top_quartile_average_weights,bottom_quartile_average_weights],axis=1)
quartile_topics.columns = ['Top Quartile','Bottom Quartile']

quartile_topics


,Top Quartile,Bottom Quartile
topic_0,0.202399,0.083830
topic_1,0.182921,0.358747
topic_2,0.260521,0.237258
topic_3,0.354159,0.320166


## Forever 21 Topic Modeling

In [64]:
reviews_df=pd.read_excel("fr21_results.xlsx")
reviews_list = reviews_df["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df['labels'] = reviews_list

(reviews_df)

,Caption,Comments,Likes,URL,labels
0,FLANNEL SZN 🍁 #F21KIDS styled by @lil_raspberr...,9,1253,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Shoe', 'White', 'Black', 'Sleeve', 'Standing..."
1,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕!\n\n💖 𝔻𝔸𝕐 ...,68,2257,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Comfort', 'Textile', 'Sleeve', 'Hou..."
2,double tap if you're obsessed with argyle prin...,13,3942,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Door', 'Neck', 'Street fashion', 'S..."
3,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕! \n\nWe wi...,41,3933,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Houseplant', 'Flowerpot', 'Green', ..."
4,Introducing 𝕂𝕀𝔻𝕊 😍 A 𝐧𝐞𝐰 collection with all c...,15,2981,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hair', 'Head', 'Smile', 'Outerwear',..."
...,...,...,...,...,...
246,🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! 🚨 𝟯𝟬% 𝗢𝗙𝗙 SHORTS! - 𝙿𝚁𝙸𝙲𝙴 𝙰𝚂 𝙼𝙰𝚁𝙺...,66,8082,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Brown', 'White', 'Textile', 'Baby & toddler ..."
247,@hangwithanna's ootd ❤️💙✨ \n\n🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! SAV...,55,19014,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Joint', 'Shoulder', 'White', 'Ne..."
248,walking into FOREVER 21’s 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺...,93,5788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sheath dress', 'Leg', 'One-piece garment', '..."
249,#INSIDELOOK @ our 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺𝙱𝙰𝙲𝙺 in ...,63,2822,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Nature', 'World', 'Font', 'Leisure', ..."


In [65]:
#checking for nulls if present any
print("Number of rows with any of the empty columns:")
print(reviews_df.isnull().sum().sum())
reviews_df=reviews_df.dropna()


restaurant_name = reviews_df.index

restaurant_name

Number of rows with any of the empty columns:
0


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            241, 242, 243, 244, 245, 246, 247, 248, 249, 250],
           dtype='int64', length=251)

In [66]:
# restaurant_name = input('provide the column name for id: ')
restaurant_review = input('provide the column name for text: ')
ntopics= input('Provide the number of latent topics: ');

provide the column name for text: labels
Provide the number of latent topics: 4


In [67]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))


def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(reviews_df[restaurant_review])

print(total_features_words.shape)

(251, 337)


/Users/allymcnulty/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [68]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
reviews_df=reviews_df.join(doc_topic)
restaurant=pd.DataFrame()

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    restaurant[topic]=reviews_df.groupby([restaurant_name])[i].mean()

restaurant=restaurant.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
restaurant.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")

INFO:lda:n_documents: 251
INFO:lda:vocab_size: 337
INFO:lda:n_words: 2924
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -22889
INFO:lda:<10> log likelihood: -16629
INFO:lda:<20> log likelihood: -16055
INFO:lda:<30> log likelihood: -15906
INFO:lda:<40> log likelihood: -15804
INFO:lda:<50> log likelihood: -15766
INFO:lda:<60> log likelihood: -15726
INFO:lda:<70> log likelihood: -15635
INFO:lda:<80> log likelihood: -15663
INFO:lda:<90> log likelihood: -15678
INFO:lda:<100> log likelihood: -15726
INFO:lda:<110> log likelihood: -15692
INFO:lda:<120> log likelihood: -15670
INFO:lda:<130> log likelihood: -15693
INFO:lda:<140> log likelihood: -15597
INFO:lda:<150> log likelihood: -15652
INFO:lda:<160> log likelihood: -15648
INFO:lda:<170> log likelihood: -15644
INFO:lda:<180> log likelihood: -15666
INFO:lda:<190> log likelihood: -15580
INFO:lda:<200> log likelihood: -15609
INFO:lda:<210> log likelihood: -15635
INFO:lda:<220> log likelihood: -15599
INFO:lda:<230> log li

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


In [69]:
## f21 topic modeling
df_topic_f21 = pd.read_excel('topic_word_dist.xlsx')
df_topic_dist_f21 = pd.read_excel('document_topic_dist.xlsx')

df_topic_f21 = df_topic_f21.rename(columns = {'Unnamed: 0': 'words', 0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

df_topic_f21_s = df_topic_f21.sort_values(by = 'topic_0', ascending = False)

df_topic_f21_s[:10]

## topic 1 body

,words,topic_0,topic_1,topic_2,topic_3
108,fashion,0.061174,0.000023,0.020363,0.083040
262,shoulder,0.058456,0.000023,0.000025,0.000016
164,joint,0.053698,0.000023,0.000025,0.000016
322,waist,0.048941,0.004627,0.000025,0.014114
171,leg,0.045543,0.000023,0.000025,0.000016
281,street,0.042144,0.000023,0.000025,0.032912
201,neck,0.040105,0.000023,0.000025,0.006282
267,sleeve,0.036707,0.000023,0.000025,0.084606
303,thigh,0.027872,0.029951,0.000025,0.000016
8,arm,0.026513,0.000023,0.000025,0.000016


In [70]:
df_topic_f21_s = df_topic_f21.sort_values(by = 'topic_1', ascending = False)

df_topic_f21_s[:10]

## topic 2 is nature

,words,topic_0,topic_1,topic_2,topic_3
230,plant,0.005444,0.069089,0.000025,0.000016
266,sky,0.000007,0.048369,0.000025,0.000016
309,top,0.000007,0.041462,0.000025,0.000016
261,shorts,0.006124,0.036858,0.000025,0.000016
312,tree,0.000007,0.036858,0.000025,0.000016
144,happy,0.000007,0.034556,0.000025,0.000016
146,hat,0.000007,0.032254,0.000025,0.017247
303,thigh,0.027872,0.029951,0.000025,0.000016
269,smile,0.013600,0.025347,0.000025,0.000016
200,nature,0.000007,0.025347,0.000025,0.000016


In [71]:
df_topic_f21_s = df_topic_f21.sort_values(by = 'topic_2', ascending = False)

df_topic_f21_s[:10]

## topic 3 is automotives

,words,topic_0,topic_1,topic_2,topic_3
84,design,0.000007,0.000023,0.063579,0.037611
16,automotive,0.000007,0.000023,0.061037,0.000016
118,font,0.000007,0.000023,0.038157,0.034478
259,shoe,0.016318,0.000023,0.033073,0.000016
306,tire,0.000007,0.000023,0.027989,0.000016
317,vehicle,0.000007,0.000023,0.027989,0.000016
183,luggage,0.000007,0.000023,0.025447,0.000016
20,bags,0.000007,0.000023,0.025447,0.000016
33,blue,0.004085,0.000023,0.022905,0.004715
94,electric,0.000007,0.000023,0.020363,0.000016


In [72]:
df_topic_f21_s = df_topic_f21.sort_values(by = 'topic_3', ascending = False)

df_topic_f21_s[:10]

## topic 4 is design

,words,topic_0,topic_1,topic_2,topic_3
267,sleeve,0.036707,0.000023,0.000025,0.084606
108,fashion,0.061174,0.000023,0.020363,0.083040
258,shirt,0.000007,0.000023,0.000025,0.054843
84,design,0.000007,0.000023,0.063579,0.037611
302,textile,0.005444,0.000023,0.000025,0.037611
118,font,0.000007,0.000023,0.038157,0.034478
281,street,0.042144,0.000023,0.000025,0.032912
330,white,0.016998,0.000023,0.000025,0.029779
105,eyewear,0.017677,0.000023,0.015278,0.026646
275,sportswear,0.000007,0.000023,0.000025,0.026646


In [73]:
df_topic_dist_f21

,index,topic_0,topic_1,topic_2,topic_3
0,0,0.008065,0.008065,0.169355,0.814516
1,1,0.009615,0.394231,0.009615,0.586538
2,2,0.975806,0.008065,0.008065,0.008065
3,3,0.008065,0.572581,0.088710,0.330645
4,4,0.973684,0.008772,0.008772,0.008772
...,...,...,...,...,...
246,246,0.008065,0.088710,0.008065,0.895161
247,247,0.875000,0.105769,0.009615,0.009615
248,248,0.701389,0.215278,0.006944,0.076389
249,249,0.008065,0.491935,0.491935,0.008065


In [74]:
reviews_df

reviews_df = reviews_df.rename(columns = {0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3
0,FLANNEL SZN 🍁 #F21KIDS styled by @lil_raspberr...,9,1253,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Shoe', 'White', 'Black', 'Sleeve', 'Standing...",0.008065,0.008065,0.169355,0.814516
1,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕!\n\n💖 𝔻𝔸𝕐 ...,68,2257,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Comfort', 'Textile', 'Sleeve', 'Hou...",0.009615,0.394231,0.009615,0.586538
2,double tap if you're obsessed with argyle prin...,13,3942,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Door', 'Neck', 'Street fashion', 'S...",0.975806,0.008065,0.008065,0.008065
3,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕! \n\nWe wi...,41,3933,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Houseplant', 'Flowerpot', 'Green', ...",0.008065,0.572581,0.088710,0.330645
4,Introducing 𝕂𝕀𝔻𝕊 😍 A 𝐧𝐞𝐰 collection with all c...,15,2981,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hair', 'Head', 'Smile', 'Outerwear',...",0.973684,0.008772,0.008772,0.008772
...,...,...,...,...,...,...,...,...,...
246,🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! 🚨 𝟯𝟬% 𝗢𝗙𝗙 SHORTS! - 𝙿𝚁𝙸𝙲𝙴 𝙰𝚂 𝙼𝙰𝚁𝙺...,66,8082,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Brown', 'White', 'Textile', 'Baby & toddler ...",0.008065,0.088710,0.008065,0.895161
247,@hangwithanna's ootd ❤️💙✨ \n\n🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! SAV...,55,19014,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Joint', 'Shoulder', 'White', 'Ne...",0.875000,0.105769,0.009615,0.009615
248,walking into FOREVER 21’s 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺...,93,5788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sheath dress', 'Leg', 'One-piece garment', '...",0.701389,0.215278,0.006944,0.076389
249,#INSIDELOOK @ our 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺𝙱𝙰𝙲𝙺 in ...,63,2822,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Nature', 'World', 'Font', 'Leisure', ...",0.008065,0.491935,0.491935,0.008065


In [75]:
reviews_df.loc[:,('comment_norm')] = reviews_df['Comments']/max(reviews_df['Comments']) #normalizing comment count
reviews_df.loc[:,('likes_norm')] = reviews_df['Likes']/max(reviews_df['Likes']) #normalizing comment count


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm
0,FLANNEL SZN 🍁 #F21KIDS styled by @lil_raspberr...,9,1253,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Shoe', 'White', 'Black', 'Sleeve', 'Standing...",0.008065,0.008065,0.169355,0.814516,0.018908,0.034220
1,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕!\n\n💖 𝔻𝔸𝕐 ...,68,2257,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Comfort', 'Textile', 'Sleeve', 'Hou...",0.009615,0.394231,0.009615,0.586538,0.142857,0.061640
2,double tap if you're obsessed with argyle prin...,13,3942,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Door', 'Neck', 'Street fashion', 'S...",0.975806,0.008065,0.008065,0.008065,0.027311,0.107658
3,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕! \n\nWe wi...,41,3933,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Houseplant', 'Flowerpot', 'Green', ...",0.008065,0.572581,0.088710,0.330645,0.086134,0.107412
4,Introducing 𝕂𝕀𝔻𝕊 😍 A 𝐧𝐞𝐰 collection with all c...,15,2981,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hair', 'Head', 'Smile', 'Outerwear',...",0.973684,0.008772,0.008772,0.008772,0.031513,0.081412
...,...,...,...,...,...,...,...,...,...,...,...
246,🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! 🚨 𝟯𝟬% 𝗢𝗙𝗙 SHORTS! - 𝙿𝚁𝙸𝙲𝙴 𝙰𝚂 𝙼𝙰𝚁𝙺...,66,8082,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Brown', 'White', 'Textile', 'Baby & toddler ...",0.008065,0.088710,0.008065,0.895161,0.138655,0.220723
247,@hangwithanna's ootd ❤️💙✨ \n\n🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! SAV...,55,19014,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Joint', 'Shoulder', 'White', 'Ne...",0.875000,0.105769,0.009615,0.009615,0.115546,0.519281
248,walking into FOREVER 21’s 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺...,93,5788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sheath dress', 'Leg', 'One-piece garment', '...",0.701389,0.215278,0.006944,0.076389,0.195378,0.158073
249,#INSIDELOOK @ our 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺𝙱𝙰𝙲𝙺 in ...,63,2822,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Nature', 'World', 'Font', 'Leisure', ...",0.008065,0.491935,0.491935,0.008065,0.132353,0.077070


In [76]:

reviews_df.loc[:,('engage_scores')] = ((0.4* reviews_df.loc[:,('likes_norm')]) + 
                                 (0.6* reviews_df.loc[:,('comment_norm')]))


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores
0,FLANNEL SZN 🍁 #F21KIDS styled by @lil_raspberr...,9,1253,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Shoe', 'White', 'Black', 'Sleeve', 'Standing...",0.008065,0.008065,0.169355,0.814516,0.018908,0.034220,0.025033
1,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕!\n\n💖 𝔻𝔸𝕐 ...,68,2257,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Comfort', 'Textile', 'Sleeve', 'Hou...",0.009615,0.394231,0.009615,0.586538,0.142857,0.061640,0.110370
2,double tap if you're obsessed with argyle prin...,13,3942,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Joint', 'Door', 'Neck', 'Street fashion', 'S...",0.975806,0.008065,0.008065,0.008065,0.027311,0.107658,0.059450
3,𝟝 𝔻𝔸𝕐𝕊 𝕆𝔽 𝕋ℝ𝔼𝔸𝕋𝕊🎃🏠❤️ 𝐂𝐎𝐌𝐌𝐄𝐍𝐓 𝘛𝘖 𝘞𝘐𝘕! \n\nWe wi...,41,3933,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Plant', 'Houseplant', 'Flowerpot', 'Green', ...",0.008065,0.572581,0.088710,0.330645,0.086134,0.107412,0.094645
4,Introducing 𝕂𝕀𝔻𝕊 😍 A 𝐧𝐞𝐰 collection with all c...,15,2981,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hair', 'Head', 'Smile', 'Outerwear',...",0.973684,0.008772,0.008772,0.008772,0.031513,0.081412,0.051473
...,...,...,...,...,...,...,...,...,...,...,...,...
246,🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! 🚨 𝟯𝟬% 𝗢𝗙𝗙 SHORTS! - 𝙿𝚁𝙸𝙲𝙴 𝙰𝚂 𝙼𝙰𝚁𝙺...,66,8082,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Brown', 'White', 'Textile', 'Baby & toddler ...",0.008065,0.088710,0.008065,0.895161,0.138655,0.220723,0.171483
247,@hangwithanna's ootd ❤️💙✨ \n\n🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! SAV...,55,19014,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Joint', 'Shoulder', 'White', 'Ne...",0.875000,0.105769,0.009615,0.009615,0.115546,0.519281,0.277040
248,walking into FOREVER 21’s 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺...,93,5788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sheath dress', 'Leg', 'One-piece garment', '...",0.701389,0.215278,0.006944,0.076389,0.195378,0.158073,0.180456
249,#INSIDELOOK @ our 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺𝙱𝙰𝙲𝙺 in ...,63,2822,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Nature', 'World', 'Font', 'Leisure', ...",0.008065,0.491935,0.491935,0.008065,0.132353,0.077070,0.110240


In [77]:
q1=np.percentile(reviews_df.engage_scores, 25) 
q2=np.percentile(reviews_df.engage_scores, 50)  
q3=np.percentile(reviews_df.engage_scores, 75)

top_quartile = reviews_df[reviews_df['engage_scores']>q3]

top_quartile.sort_values(by ='engage_scores', ascending = False).head()

top_quartile_average_weights = top_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)

top_quartile_average_weights

topic_0    0.474870
topic_1    0.177827
topic_2    0.102074
topic_3    0.245229
dtype: float64

In [78]:
bottom_quartile = reviews_df[reviews_df['engage_scores']<q1]

bottom_quartile.sort_values(by ='engage_scores', ascending = True).head()

bottom_quartile_average_weights = bottom_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)


quartile_topics = pd.concat([top_quartile_average_weights,bottom_quartile_average_weights],axis=1)
quartile_topics.columns = ['Top Quartile','Bottom Quartile']

quartile_topics

,Top Quartile,Bottom Quartile
topic_0,0.474870,0.536455
topic_1,0.177827,0.083371
topic_2,0.102074,0.169387
topic_3,0.245229,0.210787


## Gucci Topic Modeling

In [79]:
reviews_df=pd.read_excel("gucci_results.xlsx")
reviews_list = reviews_df["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df['labels'] = reviews_list

(reviews_df)

,Caption,Comments,Likes,URL,labels
0,"Part of its global tour, the Gucci Garden Arch...",86,16095,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Building', 'Font', 'Cloud', 'Gas', 'S..."
1,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats...",100,35382,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Magenta', 'Fashion design', 'Poster', 'Toy',..."
2,"@elleusa’s October cover star, @billieeilish, ...",159,100026,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hairstyle', 'Black', 'Vision care', ..."
3,The October cover of L'Officiel Hommes China f...,133,80664,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Jeans', 'Head', 'Lip', 'Sleeve', 'Wa..."
4,@chiaraferragni appears on the cover of @vogue...,214,89919,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Outerwear', 'Arm', 'One-piece garment', 'Smi..."
...,...,...,...,...,...
246,"Evoking Hollywood glamour, @jodiesmith stars i...",280,48181,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Body jewelry', 'Neck', 'Eyelash', 'Headgear'..."
247,"Launching soon on @guccibeauty, a joy-fueled f...",446,120213,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Cheek', 'Lip', 'Sky', 'Eyelash', 'Hu..."
248,"First unveiled on @guccibeauty, multi-platinum...",736,89705,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Skin', 'Lip', 'Chin', 'Hand', 'Flowe..."
249,Have you heard? @guccibeauty has a new face. S...,431,125333,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Skin', 'Lip', 'Neck', 'Carnivore', 'Gesture'..."


In [80]:
#checking for nulls if present any
print("Number of rows with any of the empty columns:")
print(reviews_df.isnull().sum().sum())
reviews_df=reviews_df.dropna()


restaurant_name = reviews_df.index

restaurant_name

Number of rows with any of the empty columns:
0


Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            241, 242, 243, 244, 245, 246, 247, 248, 249, 250],
           dtype='int64', length=251)

In [81]:
# restaurant_name = input('provide the column name for id: ')
restaurant_review = input('provide the column name for text: ')
ntopics= input('Provide the number of latent topics: ');

provide the column name for text: labels
Provide the number of latent topics: 4


In [82]:
word_tokenizer=RegexpTokenizer(r'\w+')
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_nltk=set(stopwords.words('english'))


def tokenize_text(version_desc):
    lowercase=version_desc.lower()
    text = wordnet_lemmatizer.lemmatize(lowercase)
    tokens = word_tokenizer.tokenize(text)
    return tokens

vec_words = CountVectorizer(tokenizer=tokenize_text,stop_words=stopwords_nltk,decode_error='ignore')
total_features_words = vec_words.fit_transform(reviews_df[restaurant_review])

print(total_features_words.shape)

(251, 439)


/Users/allymcnulty/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['doe', 'ha', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [83]:
model = lda.LDA(n_topics=int(ntopics), n_iter=500, random_state=1)
model.fit(total_features_words)

topic_word = model.topic_word_
doc_topic=model.doc_topic_
doc_topic=pd.DataFrame(doc_topic)
reviews_df=reviews_df.join(doc_topic)
restaurant=pd.DataFrame()

for i in range(int(ntopics)):
    topic="topic_"+str(i)
    restaurant[topic]=reviews_df.groupby([restaurant_name])[i].mean()

restaurant=restaurant.reset_index()
topics=pd.DataFrame(topic_word)
topics.columns=vec_words.get_feature_names()
topics1=topics.transpose()
print ("Topics word distribution written in file topic_word_dist.xlsx ")
topics1.to_excel("topic_word_dist.xlsx")
restaurant.to_excel("document_topic_dist.xlsx",index=False)
print ("Document topic distribution written in file document_topic_dist.xlsx ")

INFO:lda:n_documents: 251
INFO:lda:vocab_size: 439
INFO:lda:n_words: 3113
INFO:lda:n_topics: 4
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -25775
INFO:lda:<10> log likelihood: -18996
INFO:lda:<20> log likelihood: -18246
INFO:lda:<30> log likelihood: -18065
INFO:lda:<40> log likelihood: -17927
INFO:lda:<50> log likelihood: -17908
INFO:lda:<60> log likelihood: -17749
INFO:lda:<70> log likelihood: -17713
INFO:lda:<80> log likelihood: -17685
INFO:lda:<90> log likelihood: -17627
INFO:lda:<100> log likelihood: -17622
INFO:lda:<110> log likelihood: -17653
INFO:lda:<120> log likelihood: -17623
INFO:lda:<130> log likelihood: -17584
INFO:lda:<140> log likelihood: -17607
INFO:lda:<150> log likelihood: -17567
INFO:lda:<160> log likelihood: -17519
INFO:lda:<170> log likelihood: -17525
INFO:lda:<180> log likelihood: -17526
INFO:lda:<190> log likelihood: -17546
INFO:lda:<200> log likelihood: -17544
INFO:lda:<210> log likelihood: -17494
INFO:lda:<220> log likelihood: -17486
INFO:lda:<230> log li

Topics word distribution written in file topic_word_dist.xlsx 
Document topic distribution written in file document_topic_dist.xlsx 


In [84]:
## Gucci topic modeling
df_topic_gucci = pd.read_excel('topic_word_dist.xlsx')
df_topic_dist_gucci = pd.read_excel('document_topic_dist.xlsx')

df_topic_gucci = df_topic_gucci.rename(columns = {'Unnamed: 0': 'words', 0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

df_topic_gucci_s = df_topic_gucci.sort_values(by = 'topic_0', ascending = False)

df_topic_gucci_s[:10]

## topic 1 photography

,words,topic_0,topic_1,topic_2,topic_3
308,photography,0.042203,0.000015,0.000008,0.00002
154,flash,0.040748,0.000015,0.000008,0.00002
192,hair,0.033474,0.000015,0.002360,0.00002
142,eyelash,0.032020,0.001529,0.000008,0.00002
40,body,0.032020,0.001529,0.000008,0.00002
241,lip,0.029110,0.000015,0.000008,0.00002
283,one,0.027655,0.000015,0.000008,0.00002
310,piece,0.027655,0.000015,0.000008,0.00002
175,garment,0.027655,0.000015,0.000008,0.00002
268,music,0.026201,0.000015,0.000008,0.00002


In [85]:
df_topic_gucci_s = df_topic_gucci.sort_values(by = 'topic_1', ascending = False)

df_topic_gucci_s[:10]

## topic 2 is design

,words,topic_0,topic_1,topic_2,topic_3
162,font,0.000015,0.054528,0.001576,0.000020
253,material,0.000015,0.028786,0.000008,0.000020
114,design,0.000015,0.027272,0.041564,0.019728
53,building,0.000015,0.025758,0.000008,0.000020
298,pattern,0.000015,0.024243,0.000008,0.000020
435,wood,0.000015,0.024243,0.000008,0.009874
39,blue,0.000015,0.024243,0.000008,0.005932
325,property,0.000015,0.024243,0.000008,0.000020
11,art,0.010198,0.022729,0.000008,0.000020
131,electric,0.000015,0.022729,0.000008,0.003961


In [86]:
df_topic_gucci_s = df_topic_gucci.sort_values(by = 'topic_2', ascending = False)

df_topic_gucci_s[:10]

## topic 3 is fashion model/body

,words,topic_0,topic_1,topic_2,topic_3
147,fashion,0.000015,0.019700,0.130164,0.000020
362,sleeve,0.000015,0.000015,0.075279,0.000020
380,street,0.000015,0.003044,0.052541,0.000020
123,dress,0.014562,0.000015,0.044700,0.000020
422,waist,0.000015,0.000015,0.043132,0.000020
114,design,0.000015,0.027272,0.041564,0.019728
289,outerwear,0.000015,0.000015,0.025098,0.000020
353,shoe,0.000015,0.000015,0.023530,0.000020
89,collar,0.000015,0.000015,0.021178,0.000020
352,shirt,0.000015,0.000015,0.020394,0.000020


In [87]:
df_topic_gucci_s = df_topic_gucci.sort_values(by = 'topic_3', ascending = False)

df_topic_gucci_s[:10]

## topic 4 is nature

,words,topic_0,topic_1,topic_2,topic_3
313,plant,0.000015,0.000015,0.006280,0.063088
246,luggage,0.000015,0.000015,0.000008,0.043379
20,bags,0.000015,0.000015,0.000008,0.043379
19,bag,0.000015,0.000015,0.000008,0.043379
300,people,0.000015,0.000015,0.000008,0.033525
361,sky,0.000015,0.007586,0.000008,0.029583
195,happy,0.013108,0.000015,0.001576,0.027612
273,nature,0.000015,0.000015,0.000008,0.027612
418,vehicle,0.000015,0.000015,0.000008,0.023670
186,grass,0.000015,0.000015,0.000008,0.023670


In [88]:
df_topic_dist_gucci

,index,topic_0,topic_1,topic_2,topic_3
0,0,0.008772,0.973684,0.008772,0.008772
1,1,0.008065,0.411290,0.250000,0.330645
2,2,0.455224,0.007463,0.529851,0.007463
3,3,0.380597,0.007463,0.604478,0.007463
4,4,0.354167,0.006944,0.631944,0.006944
...,...,...,...,...,...
246,246,0.423611,0.076389,0.493056,0.006944
247,247,0.604478,0.007463,0.007463,0.380597
248,248,0.875000,0.009615,0.009615,0.105769
249,249,0.885965,0.096491,0.008772,0.008772


In [89]:
reviews_df

reviews_df = reviews_df.rename(columns = {0:'topic_0', 1:'topic_1', 2:'topic_2', 3:'topic_3'})

reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3
0,"Part of its global tour, the Gucci Garden Arch...",86,16095,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Building', 'Font', 'Cloud', 'Gas', 'S...",0.008772,0.973684,0.008772,0.008772
1,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats...",100,35382,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Magenta', 'Fashion design', 'Poster', 'Toy',...",0.008065,0.411290,0.250000,0.330645
2,"@elleusa’s October cover star, @billieeilish, ...",159,100026,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hairstyle', 'Black', 'Vision care', ...",0.455224,0.007463,0.529851,0.007463
3,The October cover of L'Officiel Hommes China f...,133,80664,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Jeans', 'Head', 'Lip', 'Sleeve', 'Wa...",0.380597,0.007463,0.604478,0.007463
4,@chiaraferragni appears on the cover of @vogue...,214,89919,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Outerwear', 'Arm', 'One-piece garment', 'Smi...",0.354167,0.006944,0.631944,0.006944
...,...,...,...,...,...,...,...,...,...
246,"Evoking Hollywood glamour, @jodiesmith stars i...",280,48181,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Body jewelry', 'Neck', 'Eyelash', 'Headgear'...",0.423611,0.076389,0.493056,0.006944
247,"Launching soon on @guccibeauty, a joy-fueled f...",446,120213,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Cheek', 'Lip', 'Sky', 'Eyelash', 'Hu...",0.604478,0.007463,0.007463,0.380597
248,"First unveiled on @guccibeauty, multi-platinum...",736,89705,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Skin', 'Lip', 'Chin', 'Hand', 'Flowe...",0.875000,0.009615,0.009615,0.105769
249,Have you heard? @guccibeauty has a new face. S...,431,125333,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Skin', 'Lip', 'Neck', 'Carnivore', 'Gesture'...",0.885965,0.096491,0.008772,0.008772


In [90]:
reviews_df.loc[:,('comment_norm')] = reviews_df['Comments']/max(reviews_df['Comments']) #normalizing comment count
reviews_df.loc[:,('likes_norm')] = reviews_df['Likes']/max(reviews_df['Likes']) #normalizing comment count


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm
0,"Part of its global tour, the Gucci Garden Arch...",86,16095,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Building', 'Font', 'Cloud', 'Gas', 'S...",0.008772,0.973684,0.008772,0.008772,0.018151,0.023688
1,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats...",100,35382,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Magenta', 'Fashion design', 'Poster', 'Toy',...",0.008065,0.411290,0.250000,0.330645,0.021106,0.052074
2,"@elleusa’s October cover star, @billieeilish, ...",159,100026,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hairstyle', 'Black', 'Vision care', ...",0.455224,0.007463,0.529851,0.007463,0.033558,0.147216
3,The October cover of L'Officiel Hommes China f...,133,80664,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Jeans', 'Head', 'Lip', 'Sleeve', 'Wa...",0.380597,0.007463,0.604478,0.007463,0.028071,0.118720
4,@chiaraferragni appears on the cover of @vogue...,214,89919,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Outerwear', 'Arm', 'One-piece garment', 'Smi...",0.354167,0.006944,0.631944,0.006944,0.045167,0.132341
...,...,...,...,...,...,...,...,...,...,...,...
246,"Evoking Hollywood glamour, @jodiesmith stars i...",280,48181,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Body jewelry', 'Neck', 'Eyelash', 'Headgear'...",0.423611,0.076389,0.493056,0.006944,0.059097,0.070912
247,"Launching soon on @guccibeauty, a joy-fueled f...",446,120213,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Cheek', 'Lip', 'Sky', 'Eyelash', 'Hu...",0.604478,0.007463,0.007463,0.380597,0.094133,0.176927
248,"First unveiled on @guccibeauty, multi-platinum...",736,89705,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Skin', 'Lip', 'Chin', 'Hand', 'Flowe...",0.875000,0.009615,0.009615,0.105769,0.155340,0.132026
249,Have you heard? @guccibeauty has a new face. S...,431,125333,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Skin', 'Lip', 'Neck', 'Carnivore', 'Gesture'...",0.885965,0.096491,0.008772,0.008772,0.090967,0.184462


In [91]:
reviews_df.loc[:,('engage_scores')] = ((0.4* reviews_df.loc[:,('likes_norm')]) + 
                                 (0.6* reviews_df.loc[:,('comment_norm')]))


reviews_df

,Caption,Comments,Likes,URL,labels,topic_0,topic_1,topic_2,topic_3,comment_norm,likes_norm,engage_scores
0,"Part of its global tour, the Gucci Garden Arch...",86,16095,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Building', 'Font', 'Cloud', 'Gas', 'S...",0.008772,0.973684,0.008772,0.008772,0.018151,0.023688,0.020366
1,"Juri Ueno @_juri_art_, Mari Natsuki @mari_nats...",100,35382,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Magenta', 'Fashion design', 'Poster', 'Toy',...",0.008065,0.411290,0.250000,0.330645,0.021106,0.052074,0.033493
2,"@elleusa’s October cover star, @billieeilish, ...",159,100026,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Face', 'Hairstyle', 'Black', 'Vision care', ...",0.455224,0.007463,0.529851,0.007463,0.033558,0.147216,0.079022
3,The October cover of L'Officiel Hommes China f...,133,80664,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Hair', 'Jeans', 'Head', 'Lip', 'Sleeve', 'Wa...",0.380597,0.007463,0.604478,0.007463,0.028071,0.118720,0.064330
4,@chiaraferragni appears on the cover of @vogue...,214,89919,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Outerwear', 'Arm', 'One-piece garment', 'Smi...",0.354167,0.006944,0.631944,0.006944,0.045167,0.132341,0.080036
...,...,...,...,...,...,...,...,...,...,...,...,...
246,"Evoking Hollywood glamour, @jodiesmith stars i...",280,48181,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Body jewelry', 'Neck', 'Eyelash', 'Headgear'...",0.423611,0.076389,0.493056,0.006944,0.059097,0.070912,0.063823
247,"Launching soon on @guccibeauty, a joy-fueled f...",446,120213,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Cheek', 'Lip', 'Sky', 'Eyelash', 'Hu...",0.604478,0.007463,0.007463,0.380597,0.094133,0.176927,0.127250
248,"First unveiled on @guccibeauty, multi-platinum...",736,89705,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Skin', 'Lip', 'Chin', 'Hand', 'Flowe...",0.875000,0.009615,0.009615,0.105769,0.155340,0.132026,0.146014
249,Have you heard? @guccibeauty has a new face. S...,431,125333,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Skin', 'Lip', 'Neck', 'Carnivore', 'Gesture'...",0.885965,0.096491,0.008772,0.008772,0.090967,0.184462,0.128365


In [92]:
q1=np.percentile(reviews_df.engage_scores, 25) 
q2=np.percentile(reviews_df.engage_scores, 50)  
q3=np.percentile(reviews_df.engage_scores, 75)

top_quartile = reviews_df[reviews_df['engage_scores']>q3]

top_quartile.sort_values(by ='engage_scores', ascending = False).head()

top_quartile_average_weights = top_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)

top_quartile_average_weights

topic_0    0.248888
topic_1    0.235702
topic_2    0.377049
topic_3    0.138362
dtype: float64

In [93]:
bottom_quartile = reviews_df[reviews_df['engage_scores']<q1]

bottom_quartile.sort_values(by ='engage_scores', ascending = True).head()

bottom_quartile_average_weights = bottom_quartile[["topic_0",'topic_1',"topic_2",'topic_3']].mean(axis=0)


quartile_topics = pd.concat([top_quartile_average_weights,bottom_quartile_average_weights],axis=1)
quartile_topics.columns = ['Top Quartile','Bottom Quartile']

quartile_topics

,Top Quartile,Bottom Quartile
topic_0,0.248888,0.195209
topic_1,0.235702,0.222910
topic_2,0.377049,0.442174
topic_3,0.138362,0.139707


In [101]:
zara_df=pd.read_excel("zara_results.xlsx")
hm_df=pd.read_excel("hm_results.xlsx")
fr21_df=pd.read_excel("fr21_results.xlsx")

zara_df["company"]='zara'
hm_df['company']='H&M'
fr21_df['company']='Forever 21'

frames = [zara_df, hm_df, fr21_df]

conc_df=pd.concat(frames)
conc_df

reviews_list = reviews_df["labels"].tolist()
for i in range(0,len(reviews_list)):
    reviews_list[i] = str(reviews_list[i])
reviews_df['labels'] = reviews_list

(reviews_df)



,Caption,Comments,Likes,URL,labels,company
0,The Charlotte Gainsbourg Collection for Zara. ...,230,23596,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Rectangle', 'Font', 'Technology', 'Electric ...",zara
1,The Charlotte Gainsbourg Collection for Zara....,192,48776,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Nose', 'Hair', 'Lip', 'Outerwear', 'Eyebrow'...",zara
2,The Charlotte Gainsbourg Collection for Zara....,124,17101,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Black-and-white', 'Font', 'Monochrome photog...",zara
3,ZARA ATHLETICZ. Active wear that goes faster a...,456,38788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Arm', 'Window', 'Photograph', 'Leg', 'White'...",zara
4,"ZARA ATHLETICZ. Adaptable, multifunctional des...",165,34749,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Light', 'Automotive lighting', 'Road surface...",zara
...,...,...,...,...,...,...
246,🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! 🚨 𝟯𝟬% 𝗢𝗙𝗙 SHORTS! - 𝙿𝚁𝙸𝙲𝙴 𝙰𝚂 𝙼𝙰𝚁𝙺...,66,8082,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Brown', 'White', 'Textile', 'Baby & toddler ...",Forever 21
247,@hangwithanna's ootd ❤️💙✨ \n\n🅾🅽🅻🅸🅽🅴 🅾🅽🅻🆈! SAV...,55,19014,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Clothing', 'Joint', 'Shoulder', 'White', 'Ne...",Forever 21
248,walking into FOREVER 21’s 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺...,93,5788,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sheath dress', 'Leg', 'One-piece garment', '...",Forever 21
249,#INSIDELOOK @ our 𝙐𝙇𝙏𝙄𝙈𝘼𝙏𝙀 🆂🆄🅼🅼🅴🆁 𝙺𝙸𝙲𝙺𝙱𝙰𝙲𝙺 in ...,63,2822,https://scontent-hou1-1.cdninstagram.com/v/t51...,"['Sky', 'Nature', 'World', 'Font', 'Leisure', ...",Forever 21
